In [1]:
from collections import namedtuple
from gensim.models import doc2vec
from konlpy.tag import Okt
import multiprocessing
from pprint import pprint
from gensim.models import Doc2Vec
from sklearn.linear_model import LogisticRegression
import numpy
import pickle
import numpy as np
import pandas as pd

In [2]:
twitter = Okt()

def read_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
    return data

def tokenize(doc):
  # norm, stem은 optional
  return ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]

In [3]:
import csv

output=open('Aladdin.txt','w')

with open('Aladdin.csv',"rt", encoding='utf-8') as f:
    for row in f:
        output.write(row)

In [4]:
Aladdin_raw = read_data('Aladdin.txt')
Aladdin_raw

[['text'],
 ['알라딘‥ 나랑 비교하면 너는 많은 것들을 안고 있구나.'],
 ['알라딘보러가요'],
 ["오늘 알라딘 영화본댜~!! 지금 영화관 가는 중이댜!!･:*三( o'ω')o"],
 ['"알라딘 재밌을것 같아 노래도 좋을것 같고 ㅎㅎ (@CGV 수원 - @cj_cgv in 수원시, 경기도 w/ @chacannara)"'],
 ['"로빈 윌리엄스 만큼은 아니었지만, 실사 알라딘의 지니는 너무 잘 어울리고 너무 웃기고, 아주 멋진 캐릭터였습니다 ^ㅅ^"'],
 ['이따가 알라딘 더빙판 보고온다!! 더빙판도 두근두근 기대중'],
 ['여튼 내 최애 실사영환데 알라딘도 약간 그럼. 쟈스민과 알라딘 관계 넘 좋아 ㅠㅠㅠㅠ'],
 ['아 알라딘 극장음향 너무 귀에 남아서 안떨어짐 또보고올까...ㅠㅠ'],
 ['어쨌든 알라딘 보세요. 디즈니 실사화중에 럽라도 신데렐라 만큼 개연성있다고 생각하고 각색도 조항ㅆ음. 미녀와 야수만큼 뚝뚝 끊기는 느낌도 하니고 ㅠㅠㅠ 신데렐라 담으로 내 최애 실사영화됨 ㅠㅠ'],
 ['알라딘 ost 완존 신남'],
 ['지금 자라는 아이들이 볼 자스민과 알라딘이 내가 본 것과 다르다는것만 해도 올해 개봉한 알라딘에게 후한 점수를 줄 것'],
 ['알라딘 보러간다 졸려'],
 ['"알라딘, 우리도 군사를 움직일거야. 하지만 지금은 조용히 뢰엠으로 돌아갈거야. 너와 알리바바는 네 마법을 통해 그곳으로 갈 수 있지? 바르바드로 가려면 준비할 시간이 필요해. 그리고 필요한 물건들과 재료들도 구비할게."'],
 ['알라딘 실사영화에서 자스민 공주의 시녀역 맡은 나심 페드라드 어디서 봤다 했는데 브루클린 나인나인에서 게스트 출연했었군요 '],
 ['오늘은 알라딘 담주는 기생충 가즈아'],
 ['알라딘 2차 어디가나 회전문 '],
 ['이응준 작가님의 산문집 『영혼의 무기』 이응준 이설집(異說集) 이 2017년 1월 16일 출간됐습니다. 예스24 https://goo.gl/wEJ4Nr 알라딘 https://goo.gl/fU78yN'],
 [

In [5]:
Aladdin_df = pd.DataFrame(Aladdin_raw[1:])
Aladdin_df.columns = ['text']
Aladdin_df

,text
0,알라딘‥ 나랑 비교하면 너는 많은 것들을 안고 있구나.
1,알라딘보러가요
2,오늘 알라딘 영화본댜~!! 지금 영화관 가는 중이댜!!･:*三( o'ω')o
3,"""알라딘 재밌을것 같아 노래도 좋을것 같고 ㅎㅎ (@CGV 수원 - @cj_cgv ..."
4,"""로빈 윌리엄스 만큼은 아니었지만, 실사 알라딘의 지니는 너무 잘 어울리고 너무 웃..."
5,이따가 알라딘 더빙판 보고온다!! 더빙판도 두근두근 기대중
6,여튼 내 최애 실사영환데 알라딘도 약간 그럼. 쟈스민과 알라딘 관계 넘 좋아 ㅠㅠㅠㅠ
7,아 알라딘 극장음향 너무 귀에 남아서 안떨어짐 또보고올까...ㅠㅠ
8,어쨌든 알라딘 보세요. 디즈니 실사화중에 럽라도 신데렐라 만큼 개연성있다고 생각하고...
9,알라딘 ost 완존 신남


In [6]:
Aladdin_df.drop_duplicates(subset=['text'], inplace=True)
print(Aladdin_df.isnull().values.any()) 

False


In [7]:
print(len(Aladdin_df))

3224


In [8]:
Aladdin_df.loc[Aladdin_df.text.isnull()]

,text


In [9]:
Aladdin_df = Aladdin_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(Aladdin_df.isnull().values.any()) # Null 값이 존재하는지 확인
print(len(Aladdin_df)) # 삭제 후 개수 확인

False
3224


In [10]:
Aladdin_df['text'] = Aladdin_df['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
Aladdin_df[:10]

,text
0,알라딘 나랑 비교하면 너는 많은 것들을 안고 있구나
1,알라딘보러가요
2,오늘 알라딘 영화본댜 지금 영화관 가는 중이댜
3,알라딘 재밌을것 같아 노래도 좋을것 같고 ㅎㅎ 수원 수원시 경기도
4,로빈 윌리엄스 만큼은 아니었지만 실사 알라딘의 지니는 너무 잘 어울리고 너무 웃기고...
5,이따가 알라딘 더빙판 보고온다 더빙판도 두근두근 기대중
6,여튼 내 최애 실사영환데 알라딘도 약간 그럼 쟈스민과 알라딘 관계 넘 좋아 ㅠㅠㅠㅠ
7,아 알라딘 극장음향 너무 귀에 남아서 안떨어짐 또보고올까ㅠㅠ
8,어쨌든 알라딘 보세요 디즈니 실사화중에 럽라도 신데렐라 만큼 개연성있다고 생각하고 ...
9,알라딘 완존 신남


In [11]:
# 기존에 한글이 없는 리뷰였다면 이제 더 이상 아무런 값도 없는 빈(empty) 값이 되었을 것임
Aladdin_df['text'].replace('', np.nan, inplace=True)
print(Aladdin_df.isnull().sum())
Aladdin_df.loc[Aladdin_df.text.isnull()]

text    0
dtype: int64


,text


In [12]:
Aladdin_df = Aladdin_df.dropna(how = 'any')
Aladdin_df.loc[Aladdin_df.text.isnull()]
print(len(Aladdin_df))

3224


In [13]:
Aladdin_list = Aladdin_df['text'].tolist()
Aladdin_list

['알라딘 나랑 비교하면 너는 많은 것들을 안고 있구나',
 '알라딘보러가요',
 '오늘 알라딘 영화본댜 지금 영화관 가는 중이댜 ',
 '알라딘 재밌을것 같아 노래도 좋을것 같고 ㅎㅎ  수원    수원시 경기도  ',
 '로빈 윌리엄스 만큼은 아니었지만 실사 알라딘의 지니는 너무 잘 어울리고 너무 웃기고 아주 멋진 캐릭터였습니다 ㅅ',
 '이따가 알라딘 더빙판 보고온다 더빙판도 두근두근 기대중',
 '여튼 내 최애 실사영환데 알라딘도 약간 그럼 쟈스민과 알라딘 관계 넘 좋아 ㅠㅠㅠㅠ',
 '아 알라딘 극장음향 너무 귀에 남아서 안떨어짐 또보고올까ㅠㅠ',
 '어쨌든 알라딘 보세요 디즈니 실사화중에 럽라도 신데렐라 만큼 개연성있다고 생각하고 각색도 조항ㅆ음 미녀와 야수만큼 뚝뚝 끊기는 느낌도 하니고 ㅠㅠㅠ 신데렐라 담으로 내 최애 실사영화됨 ㅠㅠ',
 '알라딘  완존 신남',
 '지금 자라는 아이들이 볼 자스민과 알라딘이 내가 본 것과 다르다는것만 해도 올해 개봉한 알라딘에게 후한 점수를 줄 것',
 '알라딘 보러간다 졸려',
 '알라딘 우리도 군사를 움직일거야 하지만 지금은 조용히 뢰엠으로 돌아갈거야 너와 알리바바는 네 마법을 통해 그곳으로 갈 수 있지 바르바드로 가려면 준비할 시간이 필요해 그리고 필요한 물건들과 재료들도 구비할게',
 '알라딘 실사영화에서 자스민 공주의 시녀역 맡은 나심 페드라드 어디서 봤다 했는데 브루클린 나인나인에서 게스트 출연했었군요 ',
 '오늘은 알라딘 담주는 기생충 가즈아',
 '알라딘 차 어디가나 회전문 ',
 '이응준 작가님의 산문집 영혼의 무기 이응준 이설집 이 년 월 일 출간됐습니다 예스  알라딘 ',
 '알라딘도 봐야되는데 샤세스 회전문 돌아야해서 보고싶은 영화 개 개봉하면 그 중 개는 못 보고 넘기는 중ㅋㅋ',
 '알라딘도 나쁘지 않았다 오히려 자파의 존재감이 제일 희미했지 역시 코브라로 변신했어야 했어 재스민 공주 아름다웠고 직접 불렀다고 하는  정말 좋았다',
 '알라딘 트레일러만 봤을 때는 그리 안 끌렸는데 막상 

In [23]:
# 형태소 분류
run_docs2 = [(tokenize(row)) for row in Aladdin_list[0:]]
print(run_docs2)

[['알라딘/Noun', '나/Noun', '랑/Josa', '비교/Noun', '하다/Verb', '너/Noun', '는/Josa', '많다/Adjective', '것/Noun', '들/Suffix', '을/Josa', '안고/Noun', '있다/Adjective'], ['알라딘/Noun', '보다/Verb'], ['오늘/Noun', '알라딘/Noun', '영화/Noun', '본/Modifier', '댜/Noun', '지금/Noun', '영화관/Noun', '가다/Verb', '중이/Noun', '댜/Noun'], ['알라딘/Noun', '재밌다/Adjective', '같다/Adjective', '노래/Noun', '도/Josa', '좋다/Adjective', '같다/Adjective', 'ㅎㅎ/KoreanParticle', '수원/Noun', '수원시/Noun', '경기도/Noun'], ['로빈/Noun', '윌리엄스/Noun', '만큼은/Josa', '아니다/Adjective', '실사/Noun', '알라딘/Noun', '의/Josa', '지니다/Verb', '너무/Adverb', '자다/Verb', '어울리다/Verb', '너무/Adverb', '웃기/Noun', '고/Josa', '아주/Noun', '멋지다/Adjective', '캐릭터/Noun', '이다/Verb', 'ㅅ/KoreanParticle'], ['이따가/Adverb', '알라딘/Noun', '더빙/Noun', '판/Noun', '보고오다/Verb', '더빙/Noun', '판도/Noun', '두근두근/Adverb', '기/Modifier', '대중/Noun'], ['여/Noun', '틀다/Verb', '내/Noun', '최애/Noun', '실사영화/Noun', '인데/Josa', '알라딘/Noun', '도/Josa', '약간/Noun', '그렇다/Adjective', '쟈스민/Noun', '과/Josa', '알라딘/Noun', '관계/Noun', '넘다/Verb', '좋다/Adjective

In [24]:
# doc2vec 에서 필요한 데이터 형식으로 변경
TaggedDocument2 = namedtuple('text', 'null')
tagged_run_docs2 = [TaggedDocument2(d) for d in run_docs2]
tagged_run_docs2

[text(null=['알라딘/Noun', '나/Noun', '랑/Josa', '비교/Noun', '하다/Verb', '너/Noun', '는/Josa', '많다/Adjective', '것/Noun', '들/Suffix', '을/Josa', '안고/Noun', '있다/Adjective']),
 text(null=['알라딘/Noun', '보다/Verb']),
 text(null=['오늘/Noun', '알라딘/Noun', '영화/Noun', '본/Modifier', '댜/Noun', '지금/Noun', '영화관/Noun', '가다/Verb', '중이/Noun', '댜/Noun']),
 text(null=['알라딘/Noun', '재밌다/Adjective', '같다/Adjective', '노래/Noun', '도/Josa', '좋다/Adjective', '같다/Adjective', 'ㅎㅎ/KoreanParticle', '수원/Noun', '수원시/Noun', '경기도/Noun']),
 text(null=['로빈/Noun', '윌리엄스/Noun', '만큼은/Josa', '아니다/Adjective', '실사/Noun', '알라딘/Noun', '의/Josa', '지니다/Verb', '너무/Adverb', '자다/Verb', '어울리다/Verb', '너무/Adverb', '웃기/Noun', '고/Josa', '아주/Noun', '멋지다/Adjective', '캐릭터/Noun', '이다/Verb', 'ㅅ/KoreanParticle']),
 text(null=['이따가/Adverb', '알라딘/Noun', '더빙/Noun', '판/Noun', '보고오다/Verb', '더빙/Noun', '판도/Noun', '두근두근/Adverb', '기/Modifier', '대중/Noun']),
 text(null=['여/Noun', '틀다/Verb', '내/Noun', '최애/Noun', '실사영화/Noun', '인데/Josa', '알라딘/Noun', '도/Josa', '약간/Noun', '그렇다

In [25]:
# load train data
doc_vectorizer = Doc2Vec.load('doc2vec.model')

# 분류를 위한 피쳐 생성
run_x = [doc_vectorizer.infer_vector(doc.null) for doc in tagged_run_docs2]
#run_y = [doc.tags[0] for doc in tagged_run_docs2]

# load the model from disk
filename = 'finalized_model.sav'

#warnings.filterwarnings("ignore", category=DeprecationWarning)

# 실제 분류 확인
loaded_model = pickle.load(open(filename, 'rb'))
print(loaded_model.predict(run_x[29].reshape(1, -1)))

['1']


/Users/domeis/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.18.2 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [26]:
sentiment = pd.DataFrame(loaded_model.predict(run_x), columns= list('t'))
sentiment

,t
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [27]:
sentiment.loc[sentiment.t=='0', 'label'] = 'negative'
sentiment.loc[sentiment.t=='1', 'label'] = 'positive'
sentiment

sentiment

,t,label
0,1,positive
1,1,positive
2,1,positive
3,1,positive
4,1,positive
5,1,positive
6,1,positive
7,1,positive
8,1,positive
9,1,positive


In [28]:
sentiment['text'] = Aladdin_df.text
sentiment

,t,label,text
0,1,positive,알라딘 나랑 비교하면 너는 많은 것들을 안고 있구나
1,1,positive,알라딘보러가요
2,1,positive,오늘 알라딘 영화본댜 지금 영화관 가는 중이댜
3,1,positive,알라딘 재밌을것 같아 노래도 좋을것 같고 ㅎㅎ 수원 수원시 경기도
4,1,positive,로빈 윌리엄스 만큼은 아니었지만 실사 알라딘의 지니는 너무 잘 어울리고 너무 웃기고...
5,1,positive,이따가 알라딘 더빙판 보고온다 더빙판도 두근두근 기대중
6,1,positive,여튼 내 최애 실사영환데 알라딘도 약간 그럼 쟈스민과 알라딘 관계 넘 좋아 ㅠㅠㅠㅠ
7,1,positive,아 알라딘 극장음향 너무 귀에 남아서 안떨어짐 또보고올까ㅠㅠ
8,1,positive,어쨌든 알라딘 보세요 디즈니 실사화중에 럽라도 신데렐라 만큼 개연성있다고 생각하고 ...
9,1,positive,알라딘 완존 신남


In [29]:
type(sentiment)

pandas.core.frame.DataFrame

In [30]:
sentiment.to_csv('aladdin_sentiment.csv', sep=',', na_rep='NaN')

In [31]:
sentiment['t'].value_counts()

1    2912
0     312
Name: t, dtype: int64